# Picking a Neighborhood for Opening a NYC Pizza Place in Chicago

## We're going to look for all the possible neighborhoods and check out the concentration of pizza places in different neighborhoods. Then we're going to overlay neighborhood area data to find out where the concentration of pizza places per area of neighborhood is the most advantageous for openining a new restaurant

In [2]:
#Importing Packages And Reading Table
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import folium
import random 
import requests
import geocoder
from geopy.geocoders import Nominatim

### We find a table with Neighborhoods in Chicago on Wikipedia and load that data. From here we're going to look up the geocoordinates of the Neighborhoods through opencage geocoder

In [3]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago')

In [4]:
table = tables[0]

In [5]:
table.head()

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [6]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
key = '35025ed897b74f248d02be04084e1aaa'
geocoder = OpenCageGeocode(key)

In [7]:
query = 'Albany Park, Chicago, IL, USA'
results = geocoder.geocode(query)
lat = results[0]['geometry']['lat']
lng = results[0]['geometry']['lng']
print ('Lat: %f Long: %f' %(lat, lng))

Lat: 41.971937 Long: -87.716174


### We're going to create an empty dataframe and loop through the neighborhoods to find the geocoordinates. <br>*From here we will create a dataframe which contains the Neighborhood  names in Chicago along with their geocoordinates for Foursquare requests later*

In [8]:
GeoNeighborhoods = pd.DataFrame(columns= ['Neighborhood', 'Latitude', 'Longitude'])

In [9]:
for num in range(0,246): #246
    Neighborhood= table['Neighborhood'][num]
    query = ('%s, Chicago, IL, USA' %Neighborhood)
    results = geocoder.geocode(query)
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']
    print ('%s Lat: %f Long: %f' %(Neighborhood, lat, lng))
    GeoNeighborhoods = GeoNeighborhoods.append({'Neighborhood': Neighborhood, 'Latitude' : lat, 'Longitude' : lng}, ignore_index=True)
    

Albany Park Lat: 41.971937 Long: -87.716174
Altgeld Gardens Lat: 41.654864 Long: -87.600446
Andersonville Lat: 41.977139 Long: -87.669273
Archer Heights Lat: 41.811422 Long: -87.726165
Armour Square Lat: 41.840033 Long: -87.633107
Ashburn Lat: 41.747533 Long: -87.711163
Ashburn Estates Lat: 41.885310 Long: -87.622130
Auburn Gresham Lat: 41.750474 Long: -87.664304
Avalon Park Lat: 41.745035 Long: -87.588658
Avondale Lat: 41.938921 Long: -87.711168
Avondale Gardens Lat: 41.938921 Long: -87.711168
Back of the Yards Lat: 41.807533 Long: -87.666163
Belmont Central Lat: 41.939796 Long: -87.653328
Belmont Gardens Lat: 41.939796 Long: -87.653328
Belmont Heights Lat: 41.940104 Long: -87.640367
Belmont Terrace Lat: 41.939796 Long: -87.653328
Beverly Lat: 41.718153 Long: -87.671767
Beverly View Lat: 41.718153 Long: -87.671767
Beverly Woods Lat: 41.683386 Long: -87.681244
Big Oaks Lat: 41.885310 Long: -87.622130
Boystown Lat: 41.943883 Long: -87.649267
Bowmanville Lat: 41.972133 Long: -87.693737
B

River's Edge Lat: 41.977042 Long: -87.735309
Riverdale Lat: 41.665809 Long: -87.605344
Rogers Park Lat: 42.010531 Long: -87.670748
Roscoe Village Lat: 41.943403 Long: -87.679142
Rosehill Lat: 41.986456 Long: -87.681199
Roseland Lat: 41.701979 Long: -87.625881
Rosemoor Lat: 41.707122 Long: -87.607219
Saint Ben's Lat: 41.885310 Long: -87.622130
Sauganash Lat: 41.990036 Long: -87.742289
Schorsch Forest View Lat: 41.947254 Long: -87.779226
Schorsch Village Lat: 41.947254 Long: -87.779226
Scottsdale Lat: 41.747440 Long: -87.739662
Sheffield Neighbors Lat: 41.910313 Long: -87.653092
Sheridan Park Lat: 41.871054 Long: -87.654807
Sheridan Station Corridor Lat: 41.998232 Long: -87.655915
Sleepy Hollow Lat: 41.814199 Long: -87.741166
Smith Park Lat: 41.892784 Long: -87.690448
South Austin Lat: 41.887876 Long: -87.764851
South Chicago Lat: 41.739134 Long: -87.549012
South Commons Lat: 41.842533 Long: -87.621995
South Deering Lat: 41.692236 Long: -87.571703
South East Ravenswood Lat: 41.965591 Lon

In [10]:
GeoNeighborhoods.head()

,Neighborhood,Latitude,Longitude
0,Albany Park,41.971937,-87.716174
1,Altgeld Gardens,41.654864,-87.600446
2,Andersonville,41.977139,-87.669273
3,Archer Heights,41.811422,-87.726165
4,Armour Square,41.840033,-87.633107


In [12]:
CLIENT_ID = 'AUD5LT2R12JI3I0I24UT3OXRGCSQEVKDLQVJ1U0VDRSPQEQX' # your Foursquare ID
CLIENT_SECRET = 'IRMAT1NTO03QCSRJARPYI3PWA2M3DSNCYBSQSCGN52S1LPSB' # your Foursquare Secret
VERSION = '20180323'
query = 'pizza'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AUD5LT2R12JI3I0I24UT3OXRGCSQEVKDLQVJ1U0VDRSPQEQX
CLIENT_SECRET:IRMAT1NTO03QCSRJARPYI3PWA2M3DSNCYBSQSCGN52S1LPSB


**We loop through our Neighborhoods and find all the Pizza places in the neighborhood**

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
Chicago_Pizza = getNearbyVenues(names=GeoNeighborhoods['Neighborhood'],
                                   latitudes=GeoNeighborhoods['Latitude'],
                                   longitudes=GeoNeighborhoods['Longitude']
                                  )

Albany Park
Altgeld Gardens
Andersonville
Archer Heights
Armour Square
Ashburn
Ashburn Estates
Auburn Gresham
Avalon Park
Avondale
Avondale Gardens
Back of the Yards
Belmont Central
Belmont Gardens
Belmont Heights
Belmont Terrace
Beverly
Beverly View
Beverly Woods
Big Oaks
Boystown
Bowmanville
Brainerd
Brickyard
Bridgeport
Brighton Park
Bronzeville
Bucktown
Budlong Woods
Buena Park
Burnside
Cabrini–Green
Calumet Heights
Canaryville
Central Station
Chatham
Chicago Lawn
Chinatown
Chrysler Village
Clarendon Park
Clearing East
Clearing West
Cottage Grove Heights
Cragin
Crestline
Dearborn Homes
Dearborn Park
Douglas Park
Dunning
East Beverly
East Chatham
East Garfield Park
East Hyde Park
East Pilsen
East Side
East Village
Eden Green
Edgebrook
Edgewater
Edgewater Beach
Edgewater Glen
Edison Park
Englewood
Fernwood
Fifth City
Ford City
Forest Glen
Fuller Park
Fulton River District
Gage Park
Galewood
The Gap
Garfield Ridge
Gladstone Park
Gold Coast
Golden Gate
Goose Island
Graceland West
Grand

## We sanity check that we're only using Pizza Places by droping all the venues that are not category Pizza Places

In [16]:
indexNames = Chicago_Pizza[ (Chicago_Pizza['Venue Category'] != 'Pizza Place')].index
Chicago_Pizza.drop(indexNames , inplace=True)
Chicago_Pizza.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Albany Park,41.971937,-87.716174,The Local Pizzeria,41.975745,-87.713269,Pizza Place
1,Albany Park,41.971937,-87.716174,Mundial Restaurant & Bar,41.968537,-87.717998,Pizza Place
3,Andersonville,41.977139,-87.669273,Ranalli's,41.978054,-87.668855,Pizza Place
4,Andersonville,41.977139,-87.669273,Calo Ristorante,41.979427,-87.668237,Pizza Place
5,Andersonville,41.977139,-87.669273,Konak Pizza and Grill,41.975984,-87.668465,Pizza Place


In [17]:
Neigh_Pizza= Chicago_Pizza.groupby('Neighborhood').count()

## Taking a count of the Pizza Places in each Neighborhood and listing them in desc. order

In [19]:
Neigh_Pizza = Neigh_Pizza.drop(columns=['Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'])

In [20]:
Neigh_Pizza = Neigh_Pizza.rename(columns={"Neighborhood Latitude": "Pizza Count"})

In [21]:
Neigh_Pizza.reset_index(level=0, inplace=True)

In [22]:
Neigh_Pizza = Neigh_Pizza.sort_values('Pizza Count', ascending=False)

In [23]:
Neigh_Pizza

,Neighborhood,Pizza Count
128,The Gap,22
73,Magnificent Mile,16
126,Streeterville,16
146,West Loop,13
144,West Lakeview,12
130,The Loop,12
104,Printer's Row,11
134,Union Ridge,11
5,Ashburn Estates,10
100,Polish Downtown,10


In [24]:
Gap = Chicago_Pizza.loc[Chicago_Pizza['Neighborhood'] == "The Gap"]

In [25]:
query = "The Gap, Chicago, IL, USA"
results = geocoder.geocode(query)
latitude = results[0]['geometry']['lat']
longitude = results[0]['geometry']['lng']
print ('Lat: %f Long: %f' %(lat, lng))

Lat: 41.947022 Long: -87.656477


In [37]:
Gapmap = folium.Map(location=[latitude, longitude],tiles='Stamen Toner', zoom_start=16)

# add markers to the map
markers_colors = []
for lat, lon, venue in zip(Gap['Venue Latitude'], Gap['Venue Longitude'], Gap['Venue']):
    label = folium.Popup(' Venue: ' + str(venue), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        color='red',
        fill_opacity=0.7).add_to(Gapmap)
       
Gapmap

# Checking to see where the concentration of Pizza Places in the Neighborhood is going to be the most advantageous for opening a new Pizza Place

### We want to find a location that contains some pizza places but provides the best area to pizza place ratio thereby giving us a large amount of people that are likely to visit the area, however not overcrowded with already established pizza places

In [43]:
#Read in Area Data for Neighborhoods
Neighz = pd.read_csv('https://raw.githubusercontent.com/Artemness/Coursera_Capstone/master/Neighborhoods_2012b.csv')

In [44]:
Neighz = Neighz.drop(columns=['the_geom', 'SEC_NEIGH', 'SHAPE_LEN'])

In [45]:
Neighz = Neighz.rename(columns={"PRI_NEIGH": "Neighborhood", "SHAPE_AREA" : 'Area'})

In [46]:
Neighz.head()

,Neighborhood,Area
0,Grand Boulevard,4.849250e+07
1,Printers Row,2.162138e+06
2,United Center,3.252051e+07
3,Sheffield & DePaul,1.048259e+07
4,Humboldt Park,1.250104e+08


In [47]:
PizzaArea = Neigh_Pizza.merge(Neighz, left_on='Neighborhood', right_on='Neighborhood')

In [48]:
PizzaArea['PizperArea'] = PizzaArea['Area']/PizzaArea['Pizza Count']

In [49]:
PizzaArea.head()

,Neighborhood,Pizza Count,Area,PizperArea
0,Magnificent Mile,16,8.824051e+05,5.515032e+04
1,Streeterville,16,1.789434e+07,1.118396e+06
2,West Loop,13,5.294506e+07,4.072697e+06
3,Wicker Park,10,2.685319e+07,2.685319e+06
4,Boystown,10,3.365779e+06,3.365779e+05


In [50]:
PizzaArea = PizzaArea.sort_values('PizperArea', ascending=False)

In [51]:
PizzaArea.drop(PizzaArea[PizzaArea['Pizza Count'] < 4].index, inplace=True)

In [52]:
PizzaArea

,Neighborhood,Pizza Count,Area,PizperArea
12,O'Hare,4,3.718356e+08,9.295890e+07
10,Lower West Side,5,8.155072e+07,1.631014e+07
5,Lake View,7,7.656118e+07,1.093731e+07
9,Rogers Park,5,5.125990e+07,1.025198e+07
7,Uptown,7,6.509564e+07,9.299378e+06
13,Old Town,4,1.984800e+07,4.962000e+06
2,West Loop,13,5.294506e+07,4.072697e+06
3,Wicker Park,10,2.685319e+07,2.685319e+06
8,Andersonville,6,9.584593e+06,1.597432e+06
11,Gold Coast,5,7.165706e+06,1.433141e+06


# We see that O'Hare is going to be the best Neighborhood to open a pizza place with Lower West Side, Lake View and Rogers Park being all close enough to be tied for a second. However O'Hare is the airport so it'll be a little difficult to establish a place there. Let's take a look at where the current locations are in the Lower West Side, Lake View and Rogers Park and see if we can choose a delightful spot for our new Pizza Place.

In [80]:
LWS = Chicago_Pizza.loc[Chicago_Pizza['Neighborhood'] == "Lower West Side"]

In [81]:
query = "Lower West Side, Chicago, IL, USA"
results = geocoder.geocode(query)
latitude = results[0]['geometry']['lat']
longitude = results[0]['geometry']['lng']
print ('Lat: %f Long: %f' %(lat, lng))

Lat: 41.943943 Long: -87.656477


In [82]:
LWS

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
382,Lower West Side,41.8542,-87.665609,Gino's East,41.853977,-87.668350,Pizza Place
383,Lower West Side,41.8542,-87.665609,Bob’s Pizza,41.854061,-87.668475,Pizza Place
384,Lower West Side,41.8542,-87.665609,Slice Factory,41.858014,-87.664029,Pizza Place
385,Lower West Side,41.8542,-87.665609,Chubby's Pizza,41.857732,-87.662405,Pizza Place
386,Lower West Side,41.8542,-87.665609,Connie's Pizza,41.852300,-87.670960,Pizza Place


In [89]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
LWSmap = folium.Map(location=[latitude, longitude], tiles='Stamen Toner', zoom_start=15)

# add markers to the map
markers_colors = []
for lat, lon, venue in zip(Chicago_Pizza['Venue Latitude'], Chicago_Pizza['Venue Longitude'], Chicago_Pizza['Venue']):
    label = folium.Popup(' Venue: ' + str(venue), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(LWSmap)
    
folium.CircleMarker(location=[41.855, -87.672],raidus = 7, color='red', popup='Possible Location', fill=True, fill_opacity=0.7).add_to(LWSmap)
       
LWSmap

## There seems to be two concentrations of Pizza places, one along West 21st Street and another by the mall along West 18th Street. However, we there might be a spot available nearby the park along West 19th Street, or along South Wood Street.

In [58]:
Lakeview = Chicago_Pizza.loc[Chicago_Pizza['Neighborhood'] == "Lake View"]

In [59]:
query = "Lake View, Chicago, IL, USA"
results = geocoder.geocode(query)
latitude = results[0]['geometry']['lat']
longitude = results[0]['geometry']['lng']
print ('Lat: %f Long: %f' %(lat, lng))

Lat: 41.943943 Long: -87.656477


In [64]:
# create map
LakeviewMap = folium.Map(location=[latitude, longitude], tiles='Stamen Toner', zoom_start=16)

# add markers to the map
markers_colors = []
for lat, lon, venue in zip(Lakeview['Venue Latitude'], Lakeview['Venue Longitude'], Lakeview['Venue']):
    label = folium.Popup(' Venue: ' + str(venue), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(LakeviewMap)
       
LakeviewMap

## In Lakeview it seems the concentration of Pizza places are along North Clark Street. We might not be able to find a suitable location that doesn't already contain a pizza place other than inside Wrigley Field and again as with O'Hare Airport, it will prove too difficult to open a pizza place inside such an establishment.

In [65]:
Rogers = Chicago_Pizza.loc[Chicago_Pizza['Neighborhood'] == "Rogers Park"]

In [66]:
query = "Rogers Park, Chicago, IL, USA"
results = geocoder.geocode(query)
latitude = results[0]['geometry']['lat']
longitude = results[0]['geometry']['lng']
print ('Lat: %f Long: %f' %(lat, lng))

Lat: 41.943943 Long: -87.656477


In [68]:
# create map
RogersMap = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to the map
markers_colors = []
for lat, lon, venue in zip(Rogers['Venue Latitude'], Rogers['Venue Longitude'], Rogers['Venue']):
    label = folium.Popup(' Venue: ' + str(venue), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(RogersMap)
       
RogersMap

## While there is some area along North Ashland Boulevard to open a pizza place, it seems that majority of traffic will arrive through North Clark Street and pass a number of pizza places along the way. Therefore we're going to go with the Lower West Side Neighborhood for opening a pizza place.

### Lets take another look at the Lower West Side location to ensure that it is where we want to open our pizza place

In [70]:
LWSmap